In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
pd.set_option("display.precision", 8)

In [2]:
trades = pd.read_csv('../temporary/trades_cleaned.csv', parse_dates = ['date'], infer_datetime_format = True)
trades

,date,tid,price,amount,Currency,nominal_amount,counter
0,2017-03-15 00:00:13,6277278,2.64900700e+04,0.07550000,MXN,2000.00028500,1
1,2017-03-15 00:00:15,100281,4.12698413e+06,0.03150000,VEF,130000.00009500,1
2,2017-03-15 00:00:15,6277265,1.29297000e+03,0.65740000,USD,849.99847800,1
3,2017-03-15 00:00:24,6277292,1.06800000e+03,0.11000000,GBP,117.48000000,1
4,2017-03-15 00:00:40,6277296,1.06293000e+03,0.11760000,GBP,125.00056800,1
...,...,...,...,...,...,...,...
42636057,2022-09-28 01:55:04,56510551,8.40412643e+07,0.00285574,COP,239999.99999785,1
42636058,2022-09-28 01:55:10,56510558,1.80295800e+04,0.03189204,USD,575.00008654,1
42636059,2022-09-28 01:55:10,56510556,2.16994700e+04,0.00553009,USD,120.00002205,1
42636060,2022-09-28 01:57:02,56510553,2.00452200e+04,0.00249436,USD,49.99999496,1


## Get Distribution of Trade Sizes

In [3]:
freq = Counter(trades['amount'])
trade_sizes = pd.DataFrame(freq.items(), columns = ['trade_size', 'frequency'])
trade_sizes.sort_values(by = 'frequency')
trade_sizes

,trade_size,frequency
0,0.07550000,106
1,0.03150000,414
2,0.65740000,1
3,0.11000000,2384
4,0.11760000,46
...,...,...
7413009,0.13503517,1
7413010,0.02979257,1
7413011,0.07398786,1
7413012,0.15726642,1


In [4]:
K = len(trades) #gives amount of total trades ~40million
trade_sizes['p'] = trade_sizes['frequency']/K #unsure if this is right way to do it, but sum(p) = 1
trade_sizes

,trade_size,frequency,p
0,0.07550000,106,0.00000249
1,0.03150000,414,0.00000971
2,0.65740000,1,0.00000002
3,0.11000000,2384,0.00005592
4,0.11760000,46,0.00000108
...,...,...,...
7413009,0.13503517,1,0.00000002
7413010,0.02979257,1,0.00000002
7413011,0.07398786,1,0.00000002
7413012,0.15726642,1,0.00000002


In [5]:
trade_sizes[trade_sizes['trade_size'] == 0.00202160]

,trade_size,frequency,p
813522,0.0020216,34,0.0000008


# Match Trades

In [3]:
def get_trades(data, window):
    
    t = str(window) + 'h'
    
    data.sort_values(['amount', 'date'], inplace=True)
    data['tid2'] = data['tid'].shift(-1)
    data['date2'] = data['date'].shift(-1)
    data['amount2'] = data['amount'].shift(-1)
    data['Currency2'] = data['Currency'].shift(-1)
    data['nominal_amount2'] = data['nominal_amount'].shift(-1)
    data['time_diff'] = data['date2'] - data['date']
    df = data[(data['time_diff'] <= (t)) & (data['amount'] == data['amount2'])]
    
    return df

In [4]:
matched_trades = get_trades(trades, 5)
matched_trades = matched_trades.sort_values('date')

In [5]:
matched_trades

,date,tid,price,amount,Currency,nominal_amount,counter,tid2,date2,amount2,Currency2,nominal_amount2,time_diff
3,2017-03-15 00:00:24,6277292,1068.00,0.11000000,GBP,117.48000000,1,100294.0,2017-03-15 00:16:23,0.11000000,VEF,456500.00000000,0 days 00:15:59
4,2017-03-15 00:00:40,6277296,1062.93,0.11760000,GBP,125.00056800,1,100435.0,2017-03-15 03:40:44,0.11760000,VEF,499999.99996800,0 days 03:40:04
7,2017-03-15 00:00:49,6277295,547460.29,0.15990000,NGN,87538.90037100,1,6277321.0,2017-03-15 00:01:37,0.15990000,USD,225.00008700,0 days 00:00:48
8,2017-03-15 00:00:58,6277291,1526.72,0.03930000,USD,60.00009600,1,6277338.0,2017-03-15 00:06:06,0.03930000,USD,60.00009600,0 days 00:05:08
11,2017-03-15 00:01:14,6277315,78740.16,0.01270000,RUB,1000.00003200,1,6277362.0,2017-03-15 00:12:30,0.01270000,RUB,1000.00003200,0 days 00:11:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42635627,2022-09-27 22:44:19,56510120,84277182.39,0.00108333,COP,91299.99999856,1,56510151.0,2022-09-27 22:52:11,0.00108333,COP,91215.30000501,0 days 00:07:52
42635745,2022-09-27 23:28:04,56510220,158900.00,0.00200000,VED,317.80000000,1,56510533.0,2022-09-28 01:42:17,0.00200000,CRC,24465.45000000,0 days 02:14:13
42635795,2022-09-27 23:46:13,56510295,1227596.37,0.00040730,RUB,500.00000150,1,56510426.0,2022-09-28 00:46:25,0.00040730,RUB,500.00000150,0 days 01:00:12
42635845,2022-09-28 00:10:07,56510348,19417017.91,0.00721017,CLP,140000.00002414,1,56510346.0,2022-09-28 00:13:07,0.00721017,CLP,140000.00002414,0 days 00:03:00


In [6]:
matched_trades.to_csv('../temporary/matched_trades.csv', index=False)

# Code Testing

In [32]:
matched_trades = matched_trades.sort_values('date')